In [5]:
import ngsolve as ngs
from ngsolve.webgui import Draw

from createGeom import makeGeo
geoOCC = makeGeo()

ngmesh = geoOCC.GenerateMesh()
mesh = ngs.Mesh(ngmesh)

# mesh.Refine()
# mesh.Refine()
# mesh.Curve(2)


In [6]:
# Draw(mesh)

In [7]:
from bhdata import BHCurves
fun_dw  = BHCurves(4)
fun_w   = fun_dw.Integrate()
fun_ddw = fun_dw.Differentiate()

mu0 = 1.256636e-6
nu0 = 1/mu0

linear = "coil_plus|coil_minus"
nonlinear = "stator"

strom = 1e5
amps = 0.025**2*3.15*strom
print(f'Applying a current of {amps:.2f} Amps')
# Draw(J, mesh)

BH data from TEAM 13 problem
Applying a current of 196.88 Amps


In [8]:
from do import solve
from copy import deepcopy

levels = 2
deg = 2

ngmesh = geoOCC.GenerateMesh()
mesh = ngs.Mesh(ngmesh)

HCurl = ngs.HCurl(mesh, order = deg, autoupdate = True, nograds = True, dirichlet = 'outer_face|coil_plus_face|coil_minus_face|stator_face')
HDiv = ngs.HDiv(mesh, order = deg, autoupdate = True)

# A = []; B = []; it = []; 
# for i in range(levels):
#     print(f"i = {i:d}")
#     # mesh.Curve(2)
    
#     J = mesh.MaterialCF({'coil_plus': (0,0,strom), 'coil_minus': (0,0,-strom), 'stator': (0,0,0)}, default = (0,0,0))

#     Ai = ngs.GridFunction(HCurl, autoupdate = True, nested = True)
#     Ai, Bi, iti = solve(HCurl,Ai,mesh,deg,J,fun_w,fun_dw,fun_ddw)

#     Bi = ngs.GridFunction(HDiv, autoupdate = True, nested = True)
#     with ngs.TaskManager(): Bi.Set(ngs.curl(Ai))

#     clipping = { "function" : True, "z" : -0.4}
#     with ngs.TaskManager(): Draw(Bi, mesh, clipping = clipping)

#     # deepcopy(Ai)

#     A.append(Ai)
#     B.append(Bi)
#     it.append(iti)

#     if i!= levels-1: mesh.Refine()

J = mesh.MaterialCF({'coil_plus': (0,0,strom), 'coil_minus': (0,0,-strom), 'stator': (0,0,0)}, default = (0,0,0))
A0 = ngs.GridFunction(HCurl)
A0, it0 = solve(HCurl,A0,mesh,deg,J,fun_w,fun_dw,fun_ddw)
B0 = ngs.GridFunction(HDiv, nested = True)
with ngs.TaskManager(): B0.Set(ngs.curl(A0), bonus_intorder=4)

clipping = { "function" : True, "z" : -0.4}
# with ngs.TaskManager(): Draw(B0, mesh, clipping = clipping)
with ngs.TaskManager(): Draw(B0, mesh, clipping = clipping)
mesh.Refine()


J = mesh.MaterialCF({'coil_plus': (0,0,strom), 'coil_minus': (0,0,-strom), 'stator': (0,0,0)}, default = (0,0,0))
A1 = ngs.GridFunction(HCurl)
A1, it1 = solve(HCurl,A1,mesh,deg,J,fun_w,fun_dw,fun_ddw)
B1 = ngs.GridFunction(HDiv, nested = True)
with ngs.TaskManager(): B1.Set(ngs.curl(A1))

clipping = { "function" : True, "z" : -0.4}
with ngs.TaskManager(): Draw(B1, mesh, clipping = clipping)
mesh.Refine()

J = mesh.MaterialCF({'coil_plus': (0,0,strom), 'coil_minus': (0,0,-strom), 'stator': (0,0,0)}, default = (0,0,0))
A2 = ngs.GridFunction(HCurl)
A2, it1 = solve(HCurl,A2,mesh,deg,J,fun_w,fun_dw,fun_ddw)
B2 = ngs.GridFunction(HDiv, nested = True)
with ngs.TaskManager(): B2.Set(ngs.curl(A2))

clipping = { "function" : True, "z" : -0.4}
with ngs.TaskManager(): Draw(B2, mesh, clipping = clipping)


# clipping = { "function" : True, "z":-0.4}
# Draw (ngs.curl(A), mesh, clipping = clipping)

Using 3D mesh with ne= 850 elements and nv= 265 points and  5153 DOFs.
 


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

Using 3D mesh with ne= 6800 elements and nv= 1592 points and  39474 DOFs.
 


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

Using 3D mesh with ne= 54400 elements and nv= 10835 points and  303628 DOFs.
 


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

In [9]:
clipping = { "function" : True, "z" : -0.4}

B0.Update()
with ngs.TaskManager(): Draw(B0, mesh, clipping = clipping)
with ngs.TaskManager(): Draw(B1, mesh, clipping = clipping)
with ngs.TaskManager(): Draw(B2, mesh, clipping = clipping)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

In [10]:
import numpy as np

err = np.zeros(levels-1)
for i in range(levels-1):
    with ngs.TaskManager():
        err[i] = ngs.Integrate((B[i+1]-B[i])**2,mesh)**(1/2)/ngs.Integrate((B[i+1])**2,mesh)**(1/2)

NameError: name 'B' is not defined

In [ ]:
print(np.log2(err[:-1]/err[1:]))
print(err)
print(it)

# [2.36188414]
# [0.13807031 0.02685977]
# [8, 8, 8]

In [ ]:
clipping = { "function" : True, "z" : -0.4}
with ngs.TaskManager():
    Draw (B[0]-B[1], mesh, clipping = clipping)



In [ ]:
!pip list

In [ ]:
clipping = { "function" : True, "z" : -0.4}

HDiv = ngs.HDiv(mesh, order = deg, dirichlet = 'outer_face|coil_plus_face|coil_minus_face|stator_face')
B0 = ngs.GridFunction(HDiv)
B0.Set(B[1])

with ngs.TaskManager():
    Draw (B[0], mesh, clipping = clipping)
    Draw (B[1], mesh, clipping = clipping)

In [ ]:
# !pip install -U webgui-jupyter-widgets

In [30]:
# help(ngs.GridFunction)

# from ngsolve import *
from netgen.occ import unit_cube

mesh = ngs.Mesh(unit_cube.GenerateMesh(maxh=0.2))
fes = ngs.HDiv(mesh, order=1, autoupdate=True)

# gf_u1 = ngs.GridFunction(fes, autoupdate=True)
gf_u1 = ngs.GridFunction(fes, autoupdate=True, nested = True)
gf_u1.Set((ngs.cos(ngs.x),ngs.sin(ngs.y),ngs.sin(ngs.y)))
# gf_u1.Set(ngs.cos(ngs.x*ngs.y))

# print("gf_u1.vec.FV().NumPy():\n", gf_u1.vec.FV().NumPy())
# mesh.Refine()
mesh.Refine()
# print("gf_u1.vec.FV().NumPy():\n", gf_u1.vec.FV().NumPy())

# Draw(ngs.div(gf_u1), mesh)

hv = gf_u1.vec.CreateVector()
hv.data = gf_u1.vec

prol = fes.Prolongation()
gf_u1.vec.Range(0,len(hv)).data = hv
prol.Prolongate(1, gf_u1.vec)


Draw(gf_u1)

AttributeError: 'NoneType' object has no attribute 'Prolongate'

In [25]:
minB1 = B1.vec.data.FV().NumPy().max()
maxB1 = B1.vec.data.FV().NumPy().min()

print(minB1, maxB1)

B1.Update()


0.0 0.0


In [24]:
help(ngs.GridFunction.Update)

Help on instancemethod in module ngsolve.comp:

Update(...)
    Update(self: ngsolve.comp.GridFunction) -> None
    
    update vector size to finite element space dimension after mesh refinement

